In [ ]:
# Define and save the static transit links

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import time

import pandas as pd

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer, SharedVehicleLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_layer_from_roads
from mnms.vehicles.veh_type import Tram, Metro, Bus, Bike
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.mobility_service.vehicle_sharing import VehicleSharingMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, load_odlayer, save_transit_links
from mnms.tools.render import draw_roads, draw_line, draw_odlayer, draw_path
from mnms.tools.geometry import points_in_polygon, get_bounding_box

In [ ]:
### Parameters

# Files and directories
current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

#coord_csv_filepath = indir + 'KV1_GVB_2609_2/Csv/POINT.csv' # file with coordinates of the network
#amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only
amsterdam_json_filepath_pt_transit = indir + "network_pt_transit.json"
transit_path = indir + "transit.json"
ams_dmd_path = indir + 'test_all_in_highway_7h_9h.csv' # dmd inside ringroad 7-9am

# Transit connection (m)
DIST_MAX = 500
DIST_CONNECTION_OD = 200
DIST_CONNECTION_PT = 100
DIST_CONNECTION_MIX = 100

In [ ]:
### Load network

mmgraph_roads = load_graph(indir + "new_network.json") # road only
mmgraph_pt = load_graph(indir + "network_pt.json")
# odlayer = load_odlayer(indir + "od_layer_clustered_200.json")
df_emoped1 = pd.read_csv(indir + 'init_pos_emoped.csv')
df_emoped2 = pd.read_csv(indir + 'init_pos_emoped.csv')

In [ ]:
### Generate layers
emoped1 = VehicleSharingMobilityService("emoped1", free_floating_possible=True, dt_matching=0)
emoped2 = VehicleSharingMobilityService("emoped2", free_floating_possible=True, dt_matching=0)
emoped_layer1 = generate_layer_from_roads(mmgraph_roads.roads, 'EMOPEDLayer1', SharedVehicleLayer, Bike, 7, [emoped1])
emoped_layer2 = generate_layer_from_roads(mmgraph_roads.roads, 'EMOPEDLayer2', SharedVehicleLayer, Bike, 7, [emoped2])
emoped_layer1.add_connected_layers(["BUSLayer", "TRAMLayer", "METROLayer"])
emoped_layer2.add_connected_layers(["BUSLayer", "TRAMLayer", "METROLayer"])

# Add stations
for nid in df_emoped1['closest_node'][:]:
    emoped1.init_free_floating_vehicles(nid, 1)
for nid in df_emoped2['closest_node'][:]:
    emoped2.init_free_floating_vehicles(nid, 1)

In [ ]:
### Connect layers
mmgraph = MultiLayerGraph([mmgraph_pt.layers["BUSLayer"], mmgraph_pt.layers["TRAMLayer"], mmgraph_pt.layers["METROLayer"],
                           emoped_layer1, emoped_layer2], odlayer)#, DIST_CONNECTION_OD)

In [ ]:
# Among PT
print('inter PT')
mmgraph.custom_connect_inter_layers(["BUSLayer", "TRAMLayer", "METROLayer"], DIST_CONNECTION_PT,
                            ensure_connect=True, max_connect_dist=DIST_MAX)
print('intra Bus')
mmgraph.custom_connect_intra_layer("BUSLayer", DIST_CONNECTION_PT, same_line=False)
print('intra Tram')
mmgraph.custom_connect_intra_layer("TRAMLayer", DIST_CONNECTION_PT, same_line=False)
print('intra Metro')
mmgraph.custom_connect_intra_layer("METROLayer", DIST_CONNECTION_PT, same_line=False)

In [ ]:
# Between PT and emoped
mmgraph.custom_connect_inter_layers(["EMOPEDLayer1", "BUSLayer"], DIST_CONNECTION_MIX,
                            ensure_connect=True, max_connect_dist=DIST_MAX)
mmgraph.custom_connect_inter_layers(["EMOPEDLayer2", "BUSLayer"], DIST_CONNECTION_MIX,
                            ensure_connect=True, max_connect_dist=DIST_MAX)
mmgraph.custom_connect_inter_layers(["EMOPEDLayer1", "TRAMLayer"], DIST_CONNECTION_MIX,
                            ensure_connect=True, max_connect_dist=DIST_MAX)
mmgraph.custom_connect_inter_layers(["EMOPEDLayer2", "TRAMLayer"], DIST_CONNECTION_MIX,
                            ensure_connect=True, max_connect_dist=DIST_MAX)
mmgraph.custom_connect_inter_layers(["EMOPEDLayer1", "METROLayer"], DIST_CONNECTION_MIX,
                            ensure_connect=True, max_connect_dist=DIST_MAX)
mmgraph.custom_connect_inter_layers(["EMOPEDLayer2", "METROLayer"], DIST_CONNECTION_MIX,
                            ensure_connect=True, max_connect_dist=DIST_MAX)

In [ ]:
### Save graph with transit
t1 = time.time()
# save_graph(mmgraph, amsterdam_json_filepath_pt_transit)
save_transit_links(mmgraph, transit_path)
t2 = time.time()
print('Files saved', t2-t1)

In [ ]:
### Stats and visual

In [ ]:
nb_layers = len(mmgraph.layers.keys())
mat_transit = np.zeros((nb_layers,nb_layers))
io=0
for olayer in mmgraph.layers.keys():
    id=0
    for dlayer in mmgraph.layers.keys():
        tr_links = mmgraph.transitlayer.links[olayer][dlayer]
        mat_transit[io,id] = len(tr_links)
        id+=1
    io+=1
        
print(mmgraph.layers.keys(),'\n', mat_transit)

In [ ]:
# Params of the visualization
colors = {'BUS': 'green', 'METRO': 'red', 'TRAM': 'skyblue'}

fig, ax = plt.subplots(figsize=(15, 15))
draw_roads(ax, mmgraph.roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

for layer in mmgraph.layers.values():
    if type(layer) == PublicTransportLayer:
        for name, line in layer.lines.items():
                draw_line(ax, mmgraph, line, color=colors[name[:name.find('_')]], 
                          linkwidth=0.4, nodesize=1, line_label=None, label_size=1, alpha=1., stopmarkeredgewidth=0.1)
print('transit links')
t1 = time.time()
tr_lines = []
for olayer in ['METROLayer']: #['METROLayer', 'TRAMLayer']:#mmgraph.layers.keys():
    for dlayer in ['METROLayer']:# ['METROLayer', 'TRAMLayer']:#mmgraph.layers.keys():
            tr_links = mmgraph.transitlayer.links[olayer][dlayer]
            for lid in tr_links:
                onid = mmgraph.graph.links[lid].downstream
                dnid = mmgraph.graph.links[lid].upstream
                onode_pos = mmgraph.graph.nodes[onid].position
                dnode_pos = mmgraph.graph.nodes[dnid].position
                tr_lines.append([onode_pos, dnode_pos])
t2 = time.time()
print('---gather lines', t2-t1)
line_segment = LineCollection(tr_lines, linestyles='-', colors='m')
ax.add_collection(line_segment)
t3 = time.time()
print('---plot lines', t3-t2)
        

legend = [Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='skyblue', linewidth=5,
            label='Tram line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='red', linewidth=5,
            label='Metro line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='green', linewidth=5,
            label='Bus line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=0, markeredgecolor='grey', markerfacecolor='grey', color='black', linewidth=1,
            label='Roads')]



legend = plt.legend(handles=legend)
legend.get_frame().set_alpha(0.92)

In [ ]:
line_segment = LineCollection(tr_lines, linestyles='-', colors='m')
ax.add_collection(line_segment)

In [ ]:
mmgraph.layers['BUSLayer'].lines['BUS_63_DIR1']['nodes']

In [ ]:
nid = 'BUS_63_DIR1_BUS_63_DIR1_7'
line = [l for l in mmgraph.layers['BUSLayer'].lines.keys() if nid in mmgraph.layers['BUSLayer'].lines[l]['nodes']][0]
print(line[:-1])


In [ ]:
mmgraph.layers['METROLayer'].lines.keys()

In [ ]:
save_transit_links